Practical, hands-on details of Internal Shortcuts in Microsoft Fabric.
Let’s break it down step-by-step so you can actually use them in a real workflow.

# 1. What is an Internal Shortcut?

An Internal Shortcut in Microsoft Fabric is a virtual reference to data stored within the same Fabric tenant’s OneLake.
It doesn’t copy the data — it simply points to the location, like a symbolic link in Windows or a mount point in Linux.

### Key concept:

Internal Shortcuts = “Access without duplication”
They save storage, time, and maintenance effort while ensuring real-time reflection of changes.

# 2. Why Use Internal Shortcuts?

Here’s why internal shortcuts are popular in Fabric:

| Problem                       | Without Shortcut                    | With Internal Shortcut   |
| ----------------------------- | ----------------------------------- | ------------------------ |
| **Duplication of large data** | Need to copy TBs between Lakehouses | Only a pointer created   |
| **Time to availability**      | Copy jobs can take hours            | Instant access           |
| **Data freshness**            | Copies can get stale                | Always reads from source |
| **Data governance**           | Multiple copies → multiple policies | One source of truth      |

Example:

Team A: Creates a cleaned customer dataset in Lakehouse A.

Team B: Needs the same dataset in Lakehouse B for BI reports.

Instead of copying → Create an internal shortcut.


# 3. Where You Can Use Internal Shortcuts

Currently supported in:

- Lakehouse (Files and Tables)

- Data Warehouse

- KQL Database

# 4. How Internal Shortcuts Work

Fabric’s OneLake stores all data in an internal namespace.
When you create a shortcut:

- It creates a metadata link in your Lakehouse/Warehouse

- The link contains the path to the original data

- When you query the shortcut, Fabric routes the request to the original data location

### Architecture Flow:

Lakehouse B → Shortcut Metadata → OneLake Path → Lakehouse A Data


#
 5. Creating an Internal Shortcut — Step-by-Step

Let’s take a practical example:

Goal:

Lakehouse B needs a shortcut to /Files/processed/sales_2025 folder in Lakehouse A.

### Step 1 — Open Target Lakehouse

In Fabric, go to Data Engineering → Lakehouse B (this is where shortcut will appear).

### Step 2 — Open Files or Tables View

Use Files tab for unstructured or parquet files.

Use Tables tab for delta tables.

### Step 3 — Create Shortcut

Click New shortcut → OneLake shortcut

Select Internal (default for same tenant)

Browse: Lakehouse A → Files → processed → sales_2025

Select folder or table.

### Step 4 — Confirm

Give the shortcut a name (good naming = clarity for others).

Click Create.

Result:
- Lakehouse B/Files/sales_data_2025 now points to Lakehouse A/Files/processed/sales_2025.

# 6. Using Internal Shortcuts in Practice

Example in PySpark (Notebook in Lakehouse B)

###### Read from shortcut folder
df = spark.read.parquet("Files/sales_data_2025")
display(df)

###### Query a shortcut table
df2 = spark.sql("SELECT * FROM shortcut_customer_table")
display(df2)

From a user’s perspective, the shortcut behaves exactly like normal data.

# 7. Behavior and Characteristics

- Real-time reflection → Changes in source = instantly available via shortcut.

- Read permissions needed → Users still need access to the source data.

- No performance penalty if source is in the same region and tenant.

- Storage footprint is negligible (only metadata).

# 8. Internal Shortcuts vs External Shortcuts
| Feature       | Internal Shortcut         | External Shortcut                 |
| ------------- | ------------------------- | --------------------------------- |
| Data Location | Same Fabric tenant        | External storage (ADLS, S3, etc.) |
| Latency       | Very low                  | Depends on external source speed  |
| Governance    | Fabric-native             | Cross-platform                    |
| Use case      | Share within organization | Bring in external data            |


# 9. Best Practices

1. Keep source data well-organized

- Shortcuts inherit the folder/table structure.

2. Set permissions at source

- Shortcuts do not override ACLs.

3. Use clear naming

- e.g., sc_sales_cleaned instead of shortcut1.

4. Avoid multi-hop shortcuts

- Don’t shortcut → shortcut → shortcut (confusing and can slow queries).

5. Document data lineage

- Helps trace back to the true source.

# 10. Limitations

- Only works within the same Fabric tenant.

- Shortcut breaks if source is deleted or moved.

- Currently not supported in all Fabric items (e.g., not directly in Pipelines as sources).

- Cannot set different permissions on shortcut than on source.

# 11. Real-World Architecture Example
     Shortcut
 
 │   Lakehouse A       │  ──────────►  Lakehouse B
 │  (Cleaned Data)     │               (BI Reports)
 └────────────────────┘
        ▲
        │
  Shortcut from
  Lakehouse C (Data Science Team)
  

A = Data Engineering team processes raw → cleaned

B = BI team builds Power BI dashboards

C = Data Science team trains ML models

Everyone uses the same single source via shortcuts.
